In [108]:
import org.json4s._
import org.json4s.jackson.JsonMethods._
import scala.runtime.ScalaRunTime._

import org.json4s._
import org.json4s.jackson.JsonMethods._
import scala.runtime.ScalaRunTime._


In [2]:
val file_path = "D:\\TU WIEN\\DIC SS20\\Assignment_2\\reviews_devset.json"
//val peopleDF = spark.read.json(path)
//peopleDF.printSchema()

file_path: String = D:\TU WIEN\DIC SS20\Assignment_2\reviews_devset.json


# Names of the columns:

root <br>
 |-- asin: string (nullable = true) <br>
 |-- category: string (nullable = true) <br>
 |-- helpful: array (nullable = true) <br>
 |    |-- element: long (containsNull = true) <br>
 |-- overall: double (nullable = true) <br>
 |-- reviewText: string (nullable = true) <br>
 |-- reviewTime: string (nullable = true) <br>
 |-- reviewerID: string (nullable = true) <br>
 |-- reviewerName: string (nullable = true) <br>
 |-- summary: string (nullable = true) <br>
 |-- unixReviewTime: long (nullable = true) <br>

In [3]:
val rdd = sc.textFile(file_path)

/* Works:
rdd.map{ row =>
  val json_row = parse(row)

  (compact(json_row \ "asin") + "test", compact(json_row \ "reviewText"))
}.collect().foreach{println _}
*/

rdd: org.apache.spark.rdd.RDD[String] = D:\TU WIEN\DIC SS20\Assignment_2\reviews_devset.json MapPartitionsRDD[1] at textFile at <console>:33


In [175]:
val rdd_text = rdd.map{ row =>
      val json_row = parse(row)

      (compact(json_row \ "reviewText"))
    }.take(10).foreach{println _}

"This was a gift for my other husband.  He's making us things from it all the time and we love the food.  Directions are simple, easy to read and interpret, and fun to make.  We all love different kinds of cuisine and Raichlen provides recipes from everywhere along the barbecue trail as he calls it. Get it and just open a page.  Have at it.  You'll love the food and it has provided us with an insight into the culture that produced it. It's all about broadening horizons.  Yum!!"
"This is a very nice spreader.  It feels very solid and the pneumatic tires give it great maneuverability and handling over bumps.  The control arm is solid metal, not a cable, which gives you precise control and will last a long time.  The settings take some experimentation with your various products to get it right, but that is true of any spreader.  It has good distribution... probably flings material a little farther on the right side than the left, but it is far more even than my crappy Edgeguard ever was."

rdd_text: Unit = ()


In [109]:
/* With array 
val rdd_text_split = rdd_text.map(x => x.split(Array('.' , ' ', '?', '!', ',', ';', ':', '(', ')', '[',']','{','}','-','_',
                                                    '`','~','#','&','*','%','$')))
*/
val rdd_text_split = rdd_text.map(x => 
"\\s+|\\,|\\\\|/|\\.|\\!|\\?|\\;|\\:|\\(|\\)|\\[|\\]|\\{|\\}|\\-|\\_|\"|\\`|\\~|\\#|\\&|\\*|\\%|\\$".r.split(x).filter(x => x.length > 1))
rdd_text_split.take(10).foreach(array => println(stringOf(array)))

Array(This, was, gift, for, my, other, husband, He's, making, us, things, from, it, all, the, time, and, we, love, the, food, Directions, are, simple, easy, to, read, and, interpret, and, fun, to, make, We, all, love, different, kinds, of, cuisine, and, Raichlen, provides, recipes, from, everywhere, along, the, barbecue, trail, as, he, calls, it, Get, it, and, just, open, page, Have, at, it, You'll, love, the, food, and, it, has, provided, us, with, an, insight, into, the, culture, that, produced, it, It's, all, about, broadening, horizons, Yum)
Array(This, is, very, nice, spreader, It, feels, very, solid, and, the, pneumatic, tires, give, it, great, maneuverability, and, handling, over, bumps, The, control, arm, is, solid, metal, not, cable, which, gives, you, precise, control, and, will, last, long, time, The, settings, take, some, experimentation, with, your, various, products, to, get, it, right, but, that, is, true, of, any, spreader, It, has, good, distribution, probably, flings,

rdd_text_split: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[13] at map at <console>:63


In [147]:
val rdd_text_no_duplicates = rdd_text_split.map(x => x.toSet.toList)

rdd_text_no_duplicates: org.apache.spark.rdd.RDD[List[String]] = MapPartitionsRDD[21] at map at <console>:64


In [148]:
val rdd_text_casefolding = rdd_text_no_duplicates.map(x => x.map(_.toLowerCase()))
rdd_text_casefolding.take(2).foreach(array => println(stringOf(array)))

List(barbecue, read, broadening, for, recipes, raichlen, insight, provided, are, it's, you'll, produced, this, different, get, all, he's, trail, page, gift, horizons, just, us, it, calls, provides, as, have, has, we, husband, open, culture, cuisine, that, to, making, was, at, kinds, my, directions, everywhere, easy, food, simple, things, he, love, with, from, fun, make, an, into, time, about, we, along, yum, other, of, and, interpret, the)
List(side, maneuverability, any, your, is, than, experimentation, solid, this, but, good, distribution, handling, pneumatic, left, it, cable, has, crappy, nice, products, feels, far, last, flings, that, arm, to, you, probably, various, was, long, gives, the, over, on, my, precise, take, spreader, farther, even, will, little, not, with, give, edgeguard, true, it, which, control, material, get, time, more, metal, very, some, great, of, settings, and, right, ever, tires, bumps, the)


rdd_text_casefolding: org.apache.spark.rdd.RDD[List[String]] = MapPartitionsRDD[22] at map at <console>:66


In [149]:
//creating stopWords List:

import scala.io.Source
import scala.collection.mutable.ListBuffer

var stopWordsHelper = new ListBuffer[String]()
val filename = "stopwords.txt"
for (line <- Source.fromFile(filename).getLines) {
    stopWordsHelper += line
}
val stopWords = stopWordsHelper.toList

import scala.io.Source
import scala.collection.mutable.ListBuffer
stopWordsHelper: scala.collection.mutable.ListBuffer[String] = ListBuffer(a's, able, about, above, according, accordingly, across, actually, after, afterwards, again, against, ain't, all, allow, allows, almost, alone, along, already, also, although, always, am, among, amongst, an, and, another, any, anybody, anyhow, anyone, anything, anyway, anyways, anywhere, apart, appear, appreciate, appropriate, are, aren't, around, as, aside, ask, asking, associated, at, available, away, awfully, be, became, because, become, becomes, becoming, been, before, beforehand, behind, being, believe, below, beside, besides, best, better, between, beyond, both, brief, but, by, c'mon, c's, came, can, can't, cannot, cant, cause, causes, certa...

In [150]:
val rdd_text_no_stopwords = rdd_text_casefolding.map(x => x.filter(!stopWords.contains(_)))
rdd_text_no_stopwords.take(2).foreach(array => println(stringOf(array)))

List(barbecue, read, broadening, recipes, raichlen, insight, provided, produced, trail, page, gift, horizons, calls, husband, open, culture, cuisine, making, kinds, directions, easy, food, simple, things, love, fun, make, time, yum, interpret)
List(side, maneuverability, experimentation, solid, good, distribution, handling, pneumatic, left, cable, crappy, nice, products, feels, flings, arm, long, precise, spreader, farther, give, edgeguard, true, control, material, time, metal, great, settings, tires, bumps)


rdd_text_no_stopwords: org.apache.spark.rdd.RDD[List[String]] = MapPartitionsRDD[23] at map at <console>:70


 # Get the amount of documents per category

In [167]:
val category_count = rdd.map{ row =>
  val json_row = parse(row)
  (compact(json_row \ "category").replace("\"", ""), 1)
}.reduceByKey((x,y) => x+y)

category_count: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[30] at reduceByKey at <console>:69


In [168]:
category_count.collect().foreach(println)

(Pet_Supplie,1235)
(Movies_and_TV,4607)
(Electronic,7825)
(Office_Product,1243)
(Musical_Instrument,500)
(Cell_Phones_and_Accessorie,3447)
(Clothing_Shoes_and_Jewelry,5749)
(Tools_and_Home_Improvement,1926)
(Baby,916)
(Patio_Lawn_and_Garde,994)
(Home_and_Kitche,4254)
(Sports_and_Outdoor,3269)
(Toys_and_Game,2253)
(Digital_Music,836)
(Apps_for_Android,2638)
(Grocery_and_Gourmet_Food,1297)
(Beauty,2023)
(Health_and_Personal_Care,2982)
(Book,22507)
(Automotive,1374)
(Kindle_Store,3205)
(CDs_and_Vinyl,3749)


In [169]:
val theMap =category_count.collectAsMap

theMap: scala.collection.Map[String,Int] = Map(Kindle_Store -> 3205, Electronic -> 7825, Automotive -> 1374, Pet_Supplie -> 1235, Clothing_Shoes_and_Jewelry -> 5749, Baby -> 916, Grocery_and_Gourmet_Food -> 1297, Musical_Instrument -> 500, Book -> 22507, Movies_and_TV -> 4607, Tools_and_Home_Improvement -> 1926, Sports_and_Outdoor -> 3269, CDs_and_Vinyl -> 3749, Apps_for_Android -> 2638, Home_and_Kitche -> 4254, Office_Product -> 1243, Health_and_Personal_Care -> 2982, Digital_Music -> 836, Cell_Phones_and_Accessorie -> 3447, Beauty -> 2023, Toys_and_Game -> 2253, Patio_Lawn_and_Garde -> 994)


In [170]:
 theMap.keys.foreach{ i =>  
 print( "Key = " + i )
 println(" Value = " + theMap(i) )}

Key = Kindle_Store Value = 3205
Key = Electronic Value = 7825
Key = Automotive Value = 1374
Key = Pet_Supplie Value = 1235
Key = Clothing_Shoes_and_Jewelry Value = 5749
Key = Baby Value = 916
Key = Grocery_and_Gourmet_Food Value = 1297
Key = Musical_Instrument Value = 500
Key = Book Value = 22507
Key = Movies_and_TV Value = 4607
Key = Tools_and_Home_Improvement Value = 1926
Key = Sports_and_Outdoor Value = 3269
Key = CDs_and_Vinyl Value = 3749
Key = Apps_for_Android Value = 2638
Key = Home_and_Kitche Value = 4254
Key = Office_Product Value = 1243
Key = Health_and_Personal_Care Value = 2982
Key = Digital_Music Value = 836
Key = Cell_Phones_and_Accessorie Value = 3447
Key = Beauty Value = 2023
Key = Toys_and_Game Value = 2253
Key = Patio_Lawn_and_Garde Value = 994


In [172]:
theMap("Kindle_Store")

res103: Int = 3205
